In [14]:
from configuration.invest_config import invest, datos_extraidos
from configuration.codigo_graficos import grafico_valor,grafico_media_ibex35, grafico_descenso_por_valor, grafico_descenso_por_valor_categoria
import numpy as np
import pandas as pd
import datetime
import os
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display
import matplotlib.pyplot as plt
from plotly.offline import iplot
from varname import nameof

In [2]:
carga_datos=datos_extraidos.datos_extraidos()
valores=carga_datos[0]
valores_name=carga_datos[1]
dic_valores=carga_datos[2]

In [3]:
#Extramos precios de apertura
dict_open={}
n=1
for i in valores:
    dict_open[n]=i["Open"]
    n+=1

In [4]:
#Sacamos la media de evolución de todos los valores del IBEX35
df_media_total=pd.DataFrame(dict_open).mean(axis=1)
df_media_IBEX35=pd.concat([dic_valores["BBVA"]["Date"], df_media_total], axis=1)
df_media_IBEX35=df_media_IBEX35.rename(columns={0:"Open"})

In [5]:
def grafico_precio_apertura(valor, dic_valores):
  caida = dic_valores[valor][(dic_valores[valor].index > 110) & (dic_valores[valor].index < 116)]
  # MEDIA DEL PRECIO DE APERTURA DEL IBEX35
  trace1 = go.Scatter(
                      x = caida['Date'],
                      y = caida['Open'],
                      name = 'citations',
                      mode= 'lines',
                      marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                      )
  data = [trace1]

  layout = dict(title = dict(text=f'PRECIO DE APERTURA {valor}', x=0.5),
              xaxis= dict(title= 'FECHA',ticklen= 5),
              yaxis= dict(title= 'PRECIO DE APERTURA',ticklen= 5)
            )

  fig = go.Figure(data = data, layout=layout)

  return(fig)

In [6]:
#Calculamos cuando tuvieron lugar el pico alto y bajo del COVID IBEX35
df_media_IBEX_2022 = df_media_IBEX35[df_media_IBEX35["Date"].str.startswith("2020")]
max_media_COVID=df_media_IBEX_2022[df_media_IBEX_2022["Open"]==df_media_IBEX_2022["Open"].max()]
min_media_COVID=df_media_IBEX_2022[df_media_IBEX_2022["Open"]==df_media_IBEX_2022["Open"].min()]

print("Pico máximo:")
print(max_media_COVID)
print("Pico mínimo:")
print(min_media_COVID)

Pico máximo:
           Date       Open
111  2020-02-17  26.617514
Pico mínimo:
           Date       Open
115  2020-03-16  17.138221


In [ ]:
#KPI caida media del IBEX 35 en el rango:
#KPI por valor de la caida promedio (o la aceleracion de caida)

In [7]:
print("Descenso de la media IBEX35=",100-(17.138221*100/26.617514))

Descenso de la media IBEX35= 35.61299150626914


In [8]:
#Calculamos cuanto descencio el valor de cada empresa entre el máximo y el mínimo de la media del IBEX35
descenso_empresa={}
for empresa in dict_max_min_value:
    try:
        descenso_empresa[empresa]=100-(int(dict_max_min_value[empresa][1])*100/int(dict_max_min_value[empresa][0]))
    except:
        pass
df_descenso_empresa = pd.DataFrame.from_dict(descenso_empresa, orient='index', columns=['value'])

In [9]:
grafico_descenso_por_valor(df_descenso_empresa)

In [10]:
descenso_medio=sum(descenso_empresa.values())/len(descenso_empresa)

empresas_poco_descenso={}

for empresa in descenso_empresa:
    if descenso_empresa[empresa]<descenso_medio:
        empresas_poco_descenso[empresa]=descenso_empresa[empresa]

In [11]:
#Empresas que descendieron menos de la media
#OBSERVAMOS QUE IBERDROLA, ANA Y GRF SON LAS QUE MENOS DESCIENDEN
empresas_poco_descenso

{'ACX': 33.33333333333333,
 'AMS': 37.5,
 'ANA': 23.63636363636364,
 'CLNX': 31.914893617021278,
 'COL': 38.46153846153846,
 'ELE': 36.0,
 'ENG': 36.0,
 'FDR': 33.33333333333333,
 'FER': 31.034482758620683,
 'GRF': 24.24242424242425,
 'IBE': 20.0,
 'ITX': 38.70967741935484,
 'LOG': 30.0,
 'RED': 31.578947368421055,
 'SAN': 33.33333333333333}

In [12]:
#Diccionario clasificacion valores por sector
sectores={"Banca":["BBVA","BKT","CABK","SAN","SAB", "UNI"],
          "Software":["CLNX","TEF","AMS","LOG"],
          "Energia":["IBE","NTGY","RED","ENG","ELE","ANE","ANA"],
          "Movilidad":["AENA","IAG","FER"],
          "Industrial":["MTS","ACX","ACS"],
          "Otros":["MAP","MEL","ITX","GRF","FDR","COL"],
          }
sectores_invertido = {valor: clave for clave, valores in sectores.items() for valor in valores}


In [13]:
lista_categoria=[]
for empresa in df_descenso_empresa.index:
    lista_categoria.append(sectores_invertido[empresa])
df_descenso_empresa["categoria"]=lista_categoria

In [14]:
grafico_descenso_por_valor_categoria(df_descenso_empresa)

In [18]:
grafico_media_ibex35(df_media_IBEX35)

In [16]:
grafico_valor("IBE", dic_valores)

In [15]:
def grafico_evolucion_volumen(valor, dic_valores):
  #Gráfico de evolución del volumen
  caida = dic_valores[valor][(dic_valores[valor].index > 110) & (dic_valores[valor].index < 116)]
  # MEDIA DEL PRECIO DE APERTURA DEL IBEX35
  trace1 = go.Scatter(
                      x = caida['Date'],
                      y = caida['Volume'],
                      name = 'citations',
                      mode= 'lines',
                      marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                      )

  data = [trace1]

  layout = dict(title = dict(text=f'VOLUMEN {valor}', x=0.5),
              xaxis= dict(title= 'FECHA',ticklen= 5),
              yaxis= dict(title= 'VOLUMEN',ticklen= 5)
            )

  fig = go.Figure(data = data, layout=layout)
  return(fig)

In [16]:
grafico_evolucion_volumen("AENA", dic_valores)

In [29]:

#Imprimimos todos los graficos de la evolucion de cada indice entre los maximo y minimo de la media en covid
for valor in valores_name:
  caida = dic_valores[valor][(dic_valores[valor].index > 110)]
  dict_max_min_value[valor]=(caida["Open"].head(1),caida["Open"].tail(1))
  # MEDIA DEL PRECIO DE APERTURA DEL IBEX35
  trace1 = go.Scatter(
                      x = caida['Date'],
                      y = caida['Open'],
                      name = 'citations',
                      mode= 'lines',
                      marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                      )

  data = [trace1]

  layout = dict(title = dict(text=f'PRECIO DE APERTURA {valor}', x=0.5),
              xaxis= dict(title= 'FECHA',ticklen= 5),
              yaxis= dict(title= 'PRECIO DE APERTURA',ticklen= 5)
            )

  fig = go.Figure(data = data, layout=layout)

  iplot(fig)